
# Lab 4: Using Access Security Settings and Policies 
## Applying Access Control Expressions (ACE) to Volumes Using ACE's and Global Security Policies 

## Security on Ezmeral Data Fabric.

Security on the Ezmearl Data Fabric is applied at the Volume which can be applied to:

- Directories and files
- Tables (if you have the full features Ezmeral Data Fabric)
- Streams (if you have the full features Ezmeral Data Fabric)

Security can be applied in two ways:

- On-demand per data type by creating an Access Control Experssion (ACE) and then applying that ACE as a security tag to the volume.
- Or through a local or global policy management which creates a catalog of security tags that can then be applied to any data type.

Below is an example we will be running through for this lab:

<p align="center">
  <img src="Pictures/edf-sec-ace-example.png" width="800" height="600" alt="Data Fabric Snapshots">                                                    
  
</p>

For more information on security features within the Ezmeral Data Fabric please see [Managing Security Policies](https://docs.datafabric.hpe.com/62/SecurityGuide/ManagingSecurityPolicies.html)

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}

echo "We are now connected to the appliance"

#
# environment variables to be adjusted/verified by the student
#these are ansible variables that are being replaced at the time of the deployment
username="student{{ STDID }}" # your Data Fabric login credentials - username  
password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
sec_username="sec-student{{ STDID }}" # your Data Fabric login credentials - username  for security comparison
sec-password="stuDISCO2020"
apiserver_host="{{ DATAFABRICAPISERVERIP }}"
ad_group="{{ DATAFABRICADGROUP }}"  # LDAP Domain Group for Data Fabric users
secvol_name="{{ DATAFABRICSECVOL }}"  # suffix for student volume name used for security comparison / user access 
cluster_name="{{ DATAFABRICCLUSTERAME }}"  # Data Fabric Cluster Name
#
# fixed environment variables setup by the HPE Data Fabrics Lab administrator - please DO NOT MODIFY!!
#these are ansible variables that are being replaced at the time of the deplaoyment

echo "your operation context is:" student{{ STDID }} "with Password:" {{ PASSSTU }}
echo "----"
echo "your test user for applying security policy context is:" sec-student{{ STDID }} "with Password:" $sec_password

## 1. Create a volume that we can apply security policies.

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster and another user to test your security policies.  Data Fabric volumes can be mounted so they are accessable from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs).  For security reasons, you can only mount volumes in the /user directory.

To understand how an ACE works on a volume with files and directories first let's create a volume:

In [ ]:
curl -k -X POST 'https://{{ DATAFABRICAPISERVERIP }}:8443/rest/volume/create?name=student{{ STDID }}-{{ DATAFABRICSECVOL }}&path=/user/student{{ STDID }}-{{ DATAFABRICSECVOL }}' --user student{{ STDID }}:{{ PASSSTU }}

## 2. Test that both users can read/write data to the created volume

- We will list out the files and create/append data to a file as the:

In [ ]:
echo "student{{ STDID }}-{{ DATAFABRICSECVOL }} security volume update $(date)" >> /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICSECVOL }}/student{{ STDID }}-security-volume-test1
ls /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICSECVOL }}
cat /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICSECVOL }}/student{{ STDID }}-security-volume-test1

## 2. Modify a volume and apply a read Access Control Expression (ACE)

A valid user or group must be present on your cluster or in your directory services.  In this case we will use the following user to apply a read only ACE to the volume:

In [ ]:
echo sec-student{{ STDID }}

In [ ]:
#echo "$username'-'$vol_name security volume update $date" >> /mapr/$cluster_name/user/$username'-'$vol_name/$username'-'security-volume-test1
#ls /mapr/$cluster_name/user/$username'-'$vol_name
#cat /mapr/$cluster_name/user/$username'-'$vol_name/$username'-'security-volume-test1
su -c "touch /mapr/mapr-cluster.etc.fr.comm.hpecorp.net/user/student1201-sec-vol1/student1201-security-volume-test1" -s /bin/sh sec-student1201

In [ ]:
#curl -k -X POST 'https://abc.sj.us:8443/rest/volume/modify?name=testVol&writeAce=g%3Agroup1%26%28%21u%3Auser1%7C%21r%3Arole1%29' --user mapr:mapr
# 'g:group1&(!u:user1|!r:role1)'
#readAce=u%3A'$sec_username'

curl -k -X POST 'https://{{ DATAFABRICAPISERVERIP }}:8443/rest/volume/modify?name=student{{ STDID }}-{{ DATAFABRICSECVOL }}&readAce=u%3A sec-student{{ STDID }}' --user student{{ STDID }}:{{ PASSSTU }}

## 2. To understand how an ACE works on a volume with files and directories first let's create a volume.




## Now what?
Congratulations! You have placed your first API calls using Postman and cURL. This simple API only accepted GET calls, so one might argue everything could have been handled by a standard browser. The advantage of using Postman is that you can save your calls and start building collections of API calls that are useful in your job. We used a very limited API to get the weather report in Paris, but the exact same technique (using Postman) could be applied with other REST APIs, such as the ones from HPE OneView, HPE SimpliVity or Redfish. 


# Time to wrap up
You can now move to the conclusion of this workshop !

* [Conclusion](2-WKSHP-Conclusion.ipynb)